#Installation Part 1
We need to install demultiplexing software.  We will get that started now so that it can run while we are doing other things

1. Open a jupyter terminal
    1. click on the jupyter *File* menu, and select *Open*.  
    2. A new browser window/tab should open, with your jupyter *home base*. Here, you should click on the *Files* tab if it is not already active
    3. Now click on *New* and select *Terminal*, which should open a new live terminal.
2. `cd ~/bioinf_nb_ngscourse2015`
3. `git pull`
4. `sudo bash install_fastq_multx.sh` # you will need to enter your password from vm-manage
5. When the installation is done, you can confirm that it worked by running `fastq-multx`, which should print help information for running the program.

In [ ]:
cd ~/bioinf_1

#More FASTQs
So far we have only been working with one FASTQ, but eventually we will want to work with all our data.  Let's set up a directory with links to all of our FASTQ files.  The `*.gz`  in the `ln -s` command means to make a link for each file in that directory that end in `.gz`

In [ ]:
%%bash
mkdir -p fastqs
ln -s /home/bitnami/test_run_data/demux_2mismatch/*.gz fastqs/

In [ ]:
ls fastqs/

#Working with Paired-Reads
So far we have only been working with a single read file, but we have paired-end read data.  If we want to use both reads, we need to do things a little bit different at some of the steps.

## Running fastq-mcf on Paired Data
It only takes two minor changes to run fastq-mcf on paired data, we need to tell it to also load the R2 file, and what to call the trimmed output from this file. 

1. neb_19_adapter.fasta 
2. fastqs/r1.GTGAAA.fastq.gz : Note we are using the original file name
2. fastqs/r2.GTGAAA.fastq.gz : NEW for paired-data
3. -q 20
4. -x 0.5
5. -o fastqs/r1.GTGAAA.trim.fastq.gz
6. -o fastqs/r2.GTGAAA.trim.fastq.gz

Let's make a directory to put the trimmed FASTQs in, then run the whole thing.

In [ ]:
%%bash
mkdir -p trimmed
fastq-mcf neb_19_adapter.fasta \
    fastqs/r1.GTGAAA.fastq.gz \
    fastqs/r2.GTGAAA.fastq.gz \
    -q 20 -x 0.5 \
    -o trimmed/r1.GTGAAA.trim.fastq.gz \
    -o trimmed/r2.GTGAAA.trim.fastq.gz

In [ ]:
ls fastqs

## Running Tophat on Paired Data
As with fastq-mcf, running Tophat on Paired Data on requires a minor change - adding the R2 file as an input; of course we will want to save the results to a different output directory.

In [ ]:
%%bash
mkdir -p th_out/GTGAAA
tophat2 -G genome/ecoli_w3110.gff \
    --library-type fr-firststrand \
    --output-dir th_out/GTGAAA \
    --max-intron-length 5 \
    --min-intron-length 4 \
    --transcriptome-max-hits 1 \
    --max-multihits 1 \
    --no-coverage-search \
    --no-novel-juncs \
    --num-threads 2 \
    genome/ecoli_w3110 \
    trimmed/r1.GTGAAA.trim.fastq.gz \
    trimmed/r2.GTGAAA.trim.fastq.gz

## Running htseq-count on Paired Data
We have to do one thing different to run htseq-count on paired data. The BAM file output by tophat will contain both reads for each spot, but by default htseq-count expects the two reads in a pair to be right next to each other in the BAM file.  By default Tophat sorts the BAM by the read's position in the genome.  We can tell htseq-count to expect this using the `--order=pos` option, but it sometimes doesn't like tophat's sorting, so it is best if we just sort it ourselves by name, and for good measure we will explicitly tell htseq-count that is what we have done with `--order=pos`.

In [ ]:
%%bash
samtools sort -n th_out/GTGAAA/accepted_hits.bam \
    th_out/GTGAAA/accepted_hits.name
htseq-count --quiet --order=name --format=bam --stranded=reverse \
    --type=gene --idattr=ID \
    th_out/GTGAAA/accepted_hits.name.bam \
    genome/ecoli_w3110.gff > counts/GTGAAA.csv

#Working with Multiple Samples
Let's kick it up another notch - we have six samples, let's run our analysis on all of them!  First let's run fastqc on everything.  This is very easy, we can just give it all the FASTQ files on the command line, and it runs on all of them.  We can use the wildcard `*` to do this simply.

In [ ]:
%%bash
mkdir -p qc_output
fastqc --threads 2 --quiet --outdir qc_output fastqs/*.fastq.gz

## A Brief journey into for loops
Most of the steps in our pipeline aren't so simple.  To apply our pipeline to multiple sample files, we need to change things in multiple places.  For example, just to run Tophat, we need to change things in four places between each run, the output directory name (twice) and the name of the FASTQ.  Doing this by hand is not only tedious, but error prone.  Doing almost the same thing repeatedly is something that computers are very good at, and people are very bad at, so let's get the computer to do the hard work.  Because the Unix shell is almost magical (it is a full fledged programming language), we can do this.  We will use a `for loop`.  This is analogous to how you would teach a child to set the table: "FOR each place at the table, put a plate . . .,
At the shell you phrase it like this:

    for PERSON in Alice Bob Carol Dave Eve
    do
    put plate at PERSON's place
    put napkin at PERSON's place
    put fork at PERSON's place
    put spoon at PERSON's place
    put knife at PERSON's place
    done

Here is a real example:

In [ ]:
%%bash
for SAMPLE in A B C D E F
    do
       echo XXXXX_${SAMPLE}_XXXXX 
    done

the `do` and `done` are essential - `do` needs to be before the "loop body" (what is going to be repeated) and `done` needs to be after it.

So let's try something almost useful:

In [ ]:
%%bash
for BARCODE in GTGAAA
    do
        echo $BARCODE
    done

###Let's run fastq-mcf in a loop:

In [ ]:
%%bash
mkdir -p trimmed

for BARCODE in GTGAAA
    do
        echo $BARCODE
        fastq-mcf neb_19_adapter.fasta \
            fastqs/r1.${BARCODE}.fastq.gz \
            fastqs/r2.${BARCODE}.fastq.gz \
            -q 20 -x 0.5 \
            -o trimmed/r1.${BARCODE}.trim.fastq.gz \
            -o trimmed/r2.${BARCODE}.trim.fastq.gz
    done

###Now let's do the same thing for tophat

In [ ]:
%%bash
for BARCODE in GTGAAA
    do
        echo $BARCODE
        mkdir -p th_out/${BARCODE}
        tophat2 -G genome/ecoli_w3110.gff \
            --library-type fr-firststrand \
            --output-dir th_out/${BARCODE} \
            --max-intron-length 5 \
            --min-intron-length 4 \
            --transcriptome-max-hits 1 \
            --max-multihits 1 \
            --no-coverage-search \
            --no-novel-juncs \
            --num-threads 2 \
            genome/ecoli_w3110 \
            trimmed/r1.${BARCODE}.trim.fastq.gz \
            trimmed/r2.${BARCODE}.trim.fastq.gz
    done

###And now for htseq-count!

In [ ]:
%%bash
for BARCODE in GTGAAA
    do
        echo $BARCODE
        samtools sort -n th_out/${BARCODE}/accepted_hits.bam \
            th_out/${BARCODE}/accepted_hits.name

        htseq-count --quiet --order=name --format=bam --stranded=reverse \
            --type=gene --idattr=ID \
            th_out/${BARCODE}/accepted_hits.name.bam \
            genome/ecoli_w3110.gff > counts/${BARCODE}.csv
    done

###Now Everything Together!!
We will now run all the samples, but first we need to generate an adapter file for all the samples. 

#### Generating a full adapter file
We still need to do the copy-and-paste part from the index primer manual, but we will do the reverse complementing automatically.  Let's do that now (if you are feeling lazy, you can use the `testrun_adapters.fasta` file in the repo directory) . . .

OK, now that it is out of the way, we need to install a python library that our reverse complementing script will use

In [ ]:
%%bash
pip install biopython

Now we can run our script.  By default it only outputs the reverse complemented sequences, but with the --both option, it will also output the original sequence.

In [ ]:
%%bash
~/bioinf_nb_ngscourse2015/revcomp.py \
    ~/bioinf_nb_ngscourse2015/testrun_adapters.fasta \
    --both --output testrun_adapters_both.fasta

####Looping over all the samples
Now we can put all of the previous commands into one big loop.  This is probably a good time for copying and pasting.  But we will make a few small changes.

1. We will add all the barcodes the the list of barcodes to iterate over
    * AGTCAA AGTTCC ATGTCA CCGTCC GTCCGC GTGAAA
2. We need to remember to use our full adapter file

In [ ]:
%%bash
mkdir -p trimmed
mkdir -p th_out
mkdir -p counts

for BARCODE in AGTCAA AGTTCC ATGTCA CCGTCC GTCCGC GTGAAA
    do
        echo $BARCODE
        fastq-mcf testrun_adapters_both.fasta \
            fastqs/r1.${BARCODE}.fastq.gz \
            fastqs/r2.${BARCODE}.fastq.gz \
            -q 20 -x 0.5 \
            -o trimmed/r1.${BARCODE}.trim.fastq.gz \
            -o trimmed/r2.${BARCODE}.trim.fastq.gz
            
        mkdir -p th_out/${BARCODE}
        tophat2 -G genome/ecoli_w3110.gff \
            --library-type fr-firststrand \
            --output-dir th_out/${BARCODE} \
            --max-intron-length 5 \
            --min-intron-length 4 \
            --transcriptome-max-hits 1 \
            --max-multihits 1 \
            --no-coverage-search \
            --no-novel-juncs \
            --num-threads 2 \
            genome/ecoli_w3110 \
            trimmed/r1.${BARCODE}.trim.fastq.gz \
            trimmed/r2.${BARCODE}.trim.fastq.gz
            
        ln th_out/${BARCODE}/accepted_hits.bam th_out/${BARCODE}.bam
        samtools index th_out/${BARCODE}.bam
        
        samtools sort -n th_out/${BARCODE}/accepted_hits.bam \
            th_out/${BARCODE}/accepted_hits.name

        htseq-count --quiet --order=name --format=bam --stranded=reverse --type=gene \
            --idattr=ID th_out/${BARCODE}/accepted_hits.name.bam \
            genome/ecoli_w3110.gff > counts/${BARCODE}.csv
    done

In [ ]:
ls th_out/